In [1]:
%load_ext line_profiler

In [2]:
from typing import List, NamedTuple, Union

from numba import njit, prange
import numpy as np
from sklearn.datasets import make_blobs

In [3]:
X, y = make_blobs(n_samples=1_000_000, n_features=100, centers=15, random_state=42)

# Naive implementation with median

In [4]:
class Leaf(NamedTuple):
    bounds: np.ndarray
    centroid: np.ndarray
    count: int = 0

KDTree = Union['Node', Leaf]
        
class Node(NamedTuple):
    pivot_value: float
    pivot_feature: int
    left: KDTree = None
    right: KDTree = None

In [5]:
def make_median_tree(X, depth: int=4) -> KDTree:
    X = np.asanyarray(X)
    if depth == 0:
        bounds = np.vstack([X.min(axis=0, keepdims=True),
                            X.max(axis=0, keepdims=True)])
        centroid = X.mean(axis=0, keepdims=True)
        return Leaf(bounds, centroid, X.shape[0])
    most_variant = X.var(axis=0).argmax()
    feature = X[:, most_variant]
    med_feature = np.median(feature)
    left = X[feature < med_feature]
    right = X[feature > med_feature]
    return Node(
        pivot_value=med_feature, pivot_feature=most_variant,
        left=make_median_tree(left, depth-1), right=make_median_tree(right, depth-1),
    )

In [6]:
%timeit make_median_tree(X)

4.41 s ± 306 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
def get_leaves(tree: KDTree) -> List[Leaf]:
    if isinstance(tree, Leaf):
        return [tree]
    return get_leaves(tree.left) + get_leaves(tree.right)

In [8]:
leaves = get_leaves(make_median_tree(X, depth=2))

In [9]:
assert len(leaves) == 4, len(leaves)

In [10]:
assert sum(l.count for l in leaves) == X.shape[0]

In [11]:
lb_approved = (leaves[0].bounds[0] <= X).all(axis=1)
ub_approved = (leaves[0].bounds[1] >= X).all(axis=1)
is_first_leave = np.logical_and(lb_approved, ub_approved)
assert is_first_leave.sum() == leaves[0].count

In [12]:
np.testing.assert_almost_equal(X[is_first_leave].mean(axis=0, keepdims=True),
                               leaves[0].centroid)

# Implementation with Otsu

In [13]:
from skimage.filters import threshold_otsu

In [14]:
thr = threshold_otsu(X[:, 0])
thr

-2.260098580327072

In [15]:
%timeit np.median(X[:, 0])

31.6 ms ± 2.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%timeit threshold_otsu(X[:, 0])

58 ms ± 1.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%timeit threshold_otsu(X[:, 0], nbins=32)

60.3 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
def make_tree(X, leaf_size: Union[int, float]=0.01, pivot=np.median) -> KDTree:
    X = np.asanyarray(X)
    if isinstance(leaf_size, float):
        if 0 <= leaf_size <= 1:
            leaf_size = int(leaf_size * X.shape[0])
        else:
            raise ValueError('leaf_size must be between 0 and 1 when float')
    if X.shape[0] < 2 * leaf_size:
        bounds = np.vstack([X.min(axis=0, keepdims=True),
                            X.max(axis=0, keepdims=True)])
        centroid = X.mean(axis=0, keepdims=True)
        return Leaf(bounds, centroid, X.shape[0])
    most_variant = X.var(axis=0).argmax()
    feature = X[:, most_variant]
    thr = pivot(feature)
    left = X[feature < thr]
    right = X[feature > thr]
    return Node(
        pivot_value=thr, pivot_feature=most_variant,
        left=make_tree(left, leaf_size=leaf_size, pivot=pivot),
        right=make_tree(right, leaf_size=leaf_size, pivot=pivot),
    )

In [19]:
%timeit make_tree(X, leaf_size=25_000, pivot=threshold_otsu)

5.01 s ± 116 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%lprun -f make_tree make_tree(X, leaf_size=25_000, pivot=threshold_otsu)

Timer unit: 1e-06 s

Total time: 4.69181 s
File: <ipython-input-18-fc5ecc726bb3>
Function: make_tree at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def make_tree(X, leaf_size: Union[int, float]=0.01, pivot=np.median) -> KDTree:
     2        61       1214.0     19.9      0.0      X = np.asanyarray(X)
     3        61        758.0     12.4      0.0      if isinstance(leaf_size, float):
     4                                                   if 0 <= leaf_size <= 1:
     5                                                       leaf_size = int(leaf_size * X.shape[0])
     6                                                   else:
     7                                                       raise ValueError('leaf_size must be between 0 and 1 when float')
     8        61        835.0     13.7      0.0      if X.shape[0] < 2 * leaf_size:
     9        31     217325.0   7010.5      4.6          bounds = np.vstack([X.mi

# Speeding up...

In [21]:
%timeit np.mean(X[:, 0])

12.7 ms ± 211 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%timeit np.min(X[:, 0]) # estymata (max - min) / średnia

11.5 ms ± 173 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
%timeit np.var(X[:, 0])

31.7 ms ± 855 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
def _most_variant_and_thr_naive(X: np.ndarray):
    mins = X.min(axis=0)
    maxes = X.max(axis=0)
    means = X.mean(axis=0)
    idx = ((maxes - mins) / means).argmax()
    return idx, means[idx]

In [25]:
%timeit _most_variant_and_thr_naive(X)

416 ms ± 1.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
@njit(parallel=True, cache=True)
def _most_variant_and_thr_manual(X: np.ndarray):
    mins = np.copy(X[0])
    maxes = np.copy(X[0])
    means = np.copy(X[0])
    for row in range(1, X.shape[0]):
        for col in prange(X.shape[1]):
            if X[row, col] < mins[col]:
                mins[col] = X[row, col]
            elif X[row, col] > maxes[col]:
                maxes[col] = X[row, col]
            means[col] += X[row, col] / X.shape[0]
    ranges = maxes - mins
    idx = (ranges / means).argmax()
    return idx, means[idx]

_most_variant_and_thr_manual(X)
None

In [59]:
%timeit _most_variant_and_thr_manual(X)

603 ms ± 4.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
class Leaf(NamedTuple):
    centroid: np.ndarray
    count: int = 0

KDTree = Union['Node', Leaf]
        
class Node(NamedTuple):
    left: KDTree = None
    right: KDTree = None

def make_tree(X, leaf_size: Union[int, float]=0.01) -> KDTree:
    X = np.asanyarray(X)
    if isinstance(leaf_size, float):
        if 0 <= leaf_size <= 1:
            leaf_size = max(int(leaf_size * X.shape[0]), 1)
        else:
            raise ValueError('leaf_size must be between 0 and 1 when float')
    if X.shape[0] < 2 * leaf_size:
        assert X.shape[0] > 0
        centroid = X.mean(axis=0, keepdims=True)
        assert not np.any(np.isnan(centroid))
        return Leaf(centroid, X.shape[0])
    most_variant, thr = _most_variant_and_thr_manual(X)
    feature = X[:, most_variant]
    left = X[feature < thr]
    right = X[feature > thr]
    return Node(
        left=make_tree(left, leaf_size=leaf_size),
        right=make_tree(right, leaf_size=leaf_size),
    )

In [29]:
%timeit make_tree(X, leaf_size=25_000)

2.93 s ± 430 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%lprun -f make_tree make_tree(X, leaf_size=25_000)

Timer unit: 1e-06 s

Total time: 2.51347 s
File: <ipython-input-28-5041b379c95f>
Function: make_tree at line 11

Line #      Hits         Time  Per Hit   % Time  Line Contents
    11                                           def make_tree(X, leaf_size: Union[int, float]=0.01) -> KDTree:
    12        71       1470.0     20.7      0.1      X = np.asanyarray(X)
    13        71        892.0     12.6      0.0      if isinstance(leaf_size, float):
    14                                                   if 0 <= leaf_size <= 1:
    15                                                       leaf_size = max(int(leaf_size * X.shape[0]), 1)
    16                                                   else:
    17                                                       raise ValueError('leaf_size must be between 0 and 1 when float')
    18        71        813.0     11.5      0.0      if X.shape[0] < 2 * leaf_size:
    19        36        481.0     13.4      0.0          assert X.shape[0] > 0
    20    

# Even simpler...

But we already have informative features in most cases! That's why we do filtering upfront!

In [66]:
def make_tree(X, leaf_size: Union[int, float]=0.01, feature_idx: int = 0) -> KDTree:
    X = np.asanyarray(X)
    if isinstance(leaf_size, float):
        if 0 <= leaf_size <= 1:
            leaf_size = max(int(leaf_size * X.shape[0]), 1)
        else:
            raise ValueError('leaf_size must be between 0 and 1 when float')
    if X.shape[0] < 2 * leaf_size:
        assert X.shape[0] > 0
        centroid = X.mean(axis=0, keepdims=True)
        assert not np.any(np.isnan(centroid))
        return Leaf(centroid, X.shape[0])
    feature = X[:, feature_idx]
    thr = np.median(feature)
    left = X[feature < thr]
    right = X[feature > thr]
    next_feature = (feature_idx + 1) % X.shape[1]
    return Node(
        left=make_tree(left, leaf_size=leaf_size, feature_idx=next_feature),
        right=make_tree(right, leaf_size=leaf_size, feature_idx=next_feature),
    )

In [67]:
%timeit make_tree(X, leaf_size=25_000)

1.91 s ± 105 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [68]:
%lprun -f make_tree make_tree(X, leaf_size=25_000)

Timer unit: 1e-06 s

Total time: 1.69091 s
File: <ipython-input-66-27abb7eb65bc>
Function: make_tree at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def make_tree(X, leaf_size: Union[int, float]=0.01, feature_idx: int = 0) -> KDTree:
     2        63       1205.0     19.1      0.1      X = np.asanyarray(X)
     3        63        847.0     13.4      0.1      if isinstance(leaf_size, float):
     4                                                   if 0 <= leaf_size <= 1:
     5                                                       leaf_size = max(int(leaf_size * X.shape[0]), 1)
     6                                                   else:
     7                                                       raise ValueError('leaf_size must be between 0 and 1 when float')
     8        63        931.0     14.8      0.1      if X.shape[0] < 2 * leaf_size:
     9        32        296.0      9.2      0.0          assert X.sh

Let's try mean as well:

In [69]:
def make_tree(X, leaf_size: Union[int, float]=0.01, feature_idx: int = 0) -> KDTree:
    X = np.asanyarray(X)
    if isinstance(leaf_size, float):
        if 0 <= leaf_size <= 1:
            leaf_size = max(int(leaf_size * X.shape[0]), 1)
        else:
            raise ValueError('leaf_size must be between 0 and 1 when float')
    if X.shape[0] < 2 * leaf_size:
        assert X.shape[0] > 0
        centroid = X.mean(axis=0, keepdims=True)
        assert not np.any(np.isnan(centroid))
        return Leaf(centroid, X.shape[0])
    feature = X[:, feature_idx]
    thr = np.mean(feature)
    left = X[feature < thr]
    right = X[feature > thr]
    next_feature = feature_idx + 1 % X.shape[1]
    return Node(
        left=make_tree(left, leaf_size=leaf_size, feature_idx=next_feature),
        right=make_tree(right, leaf_size=leaf_size, feature_idx=next_feature),
    )

In [70]:
%timeit make_tree(X, leaf_size=25_000)

1.87 s ± 117 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [71]:
%lprun -f make_tree make_tree(X, leaf_size=25_000)

Timer unit: 1e-06 s

Total time: 1.72621 s
File: <ipython-input-69-f38e9b1880e3>
Function: make_tree at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def make_tree(X, leaf_size: Union[int, float]=0.01, feature_idx: int = 0) -> KDTree:
     2        61       1279.0     21.0      0.1      X = np.asanyarray(X)
     3        61        946.0     15.5      0.1      if isinstance(leaf_size, float):
     4                                                   if 0 <= leaf_size <= 1:
     5                                                       leaf_size = max(int(leaf_size * X.shape[0]), 1)
     6                                                   else:
     7                                                       raise ValueError('leaf_size must be between 0 and 1 when float')
     8        61        842.0     13.8      0.0      if X.shape[0] < 2 * leaf_size:
     9        31        346.0     11.2      0.0          assert X.sh

# Beware!

In [ ]:
@njit(parallel=True, cache=True)
def _most_variant_and_thr_malicious(X: np.ndarray):
    mins = np.copy(X[0])
    maxes = np.copy(X[0])
    means = np.copy(X[0])
    for row in prange(1, X.shape[0]):
        for col in range(X.shape[1]):
            if X[row, col] < mins[col]:
                mins[col] = X[row, col]
            elif X[row, col] > maxes[col]:
                maxes[col] = X[row, col]
            means[col] += X[row, col] / X.shape[0]
    ranges = maxes - mins
    normalized_means = means - mins
    idx = (ranges / normalized_means).argmax()
    return idx, means[idx]

_most_variant_and_thr_malicious(X)
None

In [ ]:
%timeit _most_variant_and_thr_malicious(X)

In [ ]:
def make_tree(X, leaf_size: Union[int, float]=0.01) -> KDTree:
    X = np.asanyarray(X)
    if isinstance(leaf_size, float):
        if 0 <= leaf_size <= 1:
            leaf_size = max(int(leaf_size * X.shape[0]), 1)
        else:
            raise ValueError('leaf_size must be between 0 and 1 when float')
    if X.shape[0] < 2 * leaf_size:
        assert X.shape[0] > 0
        centroid = X.mean(axis=0, keepdims=True)
        assert not np.any(np.isnan(centroid))
        return Leaf(centroid, X.shape[0])
    most_variant, thr = _most_variant_and_thr_malicious(X)
    feature = X[:, most_variant]
    left = X[feature < thr]
    right = X[feature > thr]
    return Node(
        left=make_tree(left, leaf_size=leaf_size),
        right=make_tree(right, leaf_size=leaf_size),
    )

In [ ]:
%timeit make_tree(X, leaf_size=250_000)

In [ ]:
%lprun -f make_tree make_tree(X, leaf_size=250_000)